In [1]:
# !pip install geemap

In [2]:
!pip install earthengine-api geemap

In [1]:
import ee
import geemap

In [2]:
Map=geemap.Map()
Map

Enter verification code:  4/1ASc3gC0UJHNZpslfmoMp0QuHAZPlmOa0NQ1aw6zInm2EybpE3Y5jGZSgs90



Successfully saved authorization token.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [3]:
features = Map.draw_features
features

[]

In [4]:
point = ee.Geometry.Polygon([
    [
        [81.442192, 20.955496],
        [81.442192, 21.360202],
        [81.845861, 21.360202],
        [81.845861, 20.955496],
        [81.442192, 20.955496],
    ]
])

image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")  
    .filterBounds(point)
    .filterDate("2022-01-01", "2022-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select(["SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6","SR_B7"])
)

image = image.multiply(0.0000275).add(-0.2)

vis_params = {
    "min": 0,
    "max": 0.3,
    "bands": ["SR_B5", "SR_B4", "SR_B3"]
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")


In [5]:
props = geemap.image_props(image)
props.getInfo()

{'NOMINAL_SCALE': 30,
 'system:band_names': ['SR_B1',
  'SR_B2',
  'SR_B3',
  'SR_B4',
  'SR_B5',
  'SR_B6',
  'SR_B7']}

In [6]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In [7]:
region = Map.user_roi

In [17]:
# Make the training dataset.
training = image.sample(
    **{
        "region": region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, "training", False)
Map

Map(bottom=14691.0, center=[21.381474908405597, -3517.7456368824755], controls=(WidgetControl(options=['positi…

In [9]:
roi1 = ee.Geometry.Polygon([
    [82.436907, 21.905241],
    [82.892751, 21.588905],
    [82.502812, 21.251399],
    [82.063444, 21.588905],
    [82.436907, 21.905241]
])

In [18]:
# Make the training dataset.
training = image.sample(
    **{
        "region": roi1,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, "training", False)
Map

Map(bottom=14691.0, center=[21.381474908405597, -3517.7456368824755], controls=(WidgetControl(options=['positi…

In [11]:
# Instantiate the clusterer and train it.
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [19]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "clusters")
Map

Map(bottom=14473.0, center=[23.594194326203834, -3522.9309082031255], controls=(WidgetControl(options=['positi…

In [20]:
legend_keys = ["Green-Field","Forest","Water-Body","Sand", "Build-Up-Region"]
legend_colors = ["#2ECC71", "#145A32", "#1F77B4", "#F39C12", "#C0392B"]
legend_dict = dict(zip(legend_keys, legend_colors))

# Reclassify the map
result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(
    result, {"min": 1, "max": 5, "palette": legend_colors}, "Labelled clusters"
)

Map.add_legend(
    title="Land Cover",
    legend_dict=legend_dict,
    colors=legend_colors,
    position="bottomright"
)
Map
     

Map(bottom=14630.0, center=[22.004174972902003, -3518.756103515625], controls=(WidgetControl(options=['positio…

In [21]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

In [22]:
Map

Map(bottom=14630.0, center=[22.004174972902003, -3518.756103515625], controls=(WidgetControl(options=['positio…

In [16]:
import os

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
out_file = os.path.join(out_dir, "cluster.tif")

In [ ]:
import ee
import geemap

ee.Initialize()

roi = result.geometry()

task = ee.batch.Export.image.toDrive(
    image=result,
    description="landsat_export",
    folder="GEE",
    fileNamePrefix="Unsupervised_Result",
    region=roi,
    scale=30,
    crs="EPSG:4326",
    maxPixels=1e13
)

task.start()

print("Export started. Check status with:")
print("task.status()")
print("Or open: https://code.earthengine.google.com/tasks")


Export started. Check status with:
task.status()
Or open: https://code.earthengine.google.com/tasks
